# More Data Collection

### Strength of Schedule Data
- https://www.teamrankings.com/college-football/ranking/schedule-strength-by-other

In [5]:
# Get general dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# Generate dates to grab strength of schedules
from datetime import date, timedelta

start_date = date(2005, 5, 1)
end_date = date(2024, 5, 1)

date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date)
    current_date = current_date.replace(year=current_date.year + 1)

# Convert dates to strings in yyyy-mm-dd format for urls
date_strings = [d.strftime('%Y-%m-%d') for d in date_list]


In [111]:
# Generate list of urls
base_url = 'https://www.teamrankings.com/college-football/ranking/schedule-strength-by-other?date='
urls = [base_url + date for date in date_strings]
urls[:2]

['https://www.teamrankings.com/college-football/ranking/schedule-strength-by-other?date=2005-05-01',
 'https://www.teamrankings.com/college-football/ranking/schedule-strength-by-other?date=2006-05-01']

In [116]:
# Set up bs4 for scraping
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

years, ranks, teams, ratings = [], [], [], []
schedule_strengths = pd.DataFrame()
# Get all tables using loop
for url in urls:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')[0]
    entries = table.find_all('td')

    for i, entry in enumerate(entries):
        # Add all of the entries
        n = i % 6
        try:
            if n == 0:
                ranks.append(entry.text)
            elif n == 1:
                teams.append(re.findall(r'^(.*?)\(', entry.text)[0].strip())
            elif n == 2:
                ratings.append(entry.text)
            elif n == 3:
                # Add the year
                years.append(url[-10:-6])
            else:
                continue
        except:
            break
    
schedule_strengths_df = pd.DataFrame({'year': years, 'rank': ranks, 'team': teams, 'rating': ratings})
schedule_strengths_df.head()    

,year,rank,team,rating
0,2005,1,USC,8.4
1,2005,2,Miami,6.8
2,2005,3,Oregon St,6.7
3,2005,4,Oklahoma,6.4
4,2005,5,Arizona St,6.4


In [119]:
schedule_strengths_df.tail()  

,year,rank,team,rating
2494,2024,129,Florida Intl,-11.3
2495,2024,130,N Illinois,-11.4
2496,2024,131,E Michigan,-13.3
2497,2024,132,Akron,-13.3
2498,2024,133,Kent St,-13.9


In [118]:
schedule_strengths_df.to_csv('data/schedule_strengths.csv')